In [ ]:
# default_exp cci

# Storm detection

> finds low pressure systems

In [ ]:
#export
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#export
from drcstools.derivative import *


def stormdetection(stormobj, slpname = "slp", pmax = 100000):
    
    stormtrack.derivative(slpname, N = 10, dim = stormobj._get_name_longitude())
    stormtrack.derivative(slpname, N = 10, dim = stormobj._get_name_latitude())
        
        
    zero_crossings_x = xr.where(xr.ufuncs.signbit(stormtrack.ds.ddlonslp).astype(int).diff("lon") != 0, 1, 0)
    zero_crossings_y = xr.where(xr.ufuncs.signbit(stormtrack.ds.ddlatslp).astype(int).diff("lat") != 0, 1, 0)
    
    zero_crossings = xr.where((zero_crossings_x == 1) & (zero_crossings_y == 1), 1, 0)
    
    dP = 0.5 * (stormobj.ds.ddlonslp + stormobj.ds.ddlatslp)
    Pzonal = dP.mean([stormobj._get_name_time(), stormobj._get_name_latitude()])

    lows = stormobj.ds[slpname].where(zero_crossings == 1).where(dP - Pzonal > 1)
    lows = lows.where(lows < 100000)
    return lows

# Examples

In [ ]:
ds = xr.open_dataset("data/rca_unrot_data_1_5.nc", use_cftime = True)
ds = ds.rename({"rlon":"lon", "rlat":"lat"})

In [ ]:
stormtrack = storm(ds = ds)
stormtrack.set_up(force = True)

NameError: name 'storm' is not defined

In [ ]:
lows = stormdetection(stormtrack)